In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
drive.mount('/content/drive')
filename_train = '/content/drive/My Drive/LaboDatos2022/SegundaEjercitacion/train.csv'
filename_test = '/content/drive/My Drive/LaboDatos2022/SegundaEjercitacion/test4alumnxs.csv'

df_train = pd.read_csv(filename_train)
df_test = pd.read_csv(filename_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def arreglar_dia(x):
  if x == 'Day':
    return '1'
  else:
    return '0'

def arreglar_descripcion(x):
  x=x.lower()
  if 'closed' in x:
    return '1'
  else:
    return '0'


In [ ]:
y = np.array(df_train['Severity'])

columnas_clima = ['Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)','Visibility(mi)',  'Wind_Speed(mph)','Precipitation(in)']

columnas_lugar = ['Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop', 'Traffic_Calming','Traffic_Signal','Turning_Loop']

columnas_dia = ['Sunrise_Sunset','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight']

columnas_descripcion = ['Description']


columnas_varias = columnas_dia + columnas_clima + columnas_lugar



In [ ]:
print(columnas_varias)

['Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']


In [ ]:
print(df_train['Sunrise_Sunset'][0])
print(df_train[columnas_lugar])

0
      Sunrise_Sunset Civil_Twilight Nautical_Twilight Astronomical_Twilight
0                  0              0                 0                     0
1                  0              0                 0                     0
2                  0              0                 0                     0
3                  1              1                 1                     1
4                  1              1                 1                     1
...              ...            ...               ...                   ...
45829              1              1                 1                     1
45830              0              0                 0                     0
45831              0              0                 0                     0
45832              1              1                 1                     1
45833              0              1                 1                     1

[45834 rows x 4 columns]


In [ ]:
#df_train['Sunrise_Sunset'].unique()
#df_train[columnas_dia]=df_train[columnas_dia].apply(arreglar_dia)
df_train['Sunrise_Sunset'] = df_train['Sunrise_Sunset'].apply(arreglar_dia)
df_train['Civil_Twilight'] = df_train['Civil_Twilight'].apply(arreglar_dia)
df_train['Nautical_Twilight'] = df_train['Nautical_Twilight'].apply(arreglar_dia)
df_train['Astronomical_Twilight'] = df_train['Astronomical_Twilight'].apply(arreglar_dia)

In [ ]:
print(df_train[columnas_descripcion])
df_train['Description']=df_train['Description'].apply(arreglar_descripcion)
print(df_train[columnas_descripcion])

                                             Description
0      Stationary traffic on US-70 W from Lynn Rd (US...
1      52500 LONOAK RD. CA TOW ADVISED ETA 30MIN DRIV...
2      Incident on HOLLYWOOD ST near WINCHESTER AVE D...
3      Stationary traffic on FL-45 - County Hwy-876 -...
4      Incident on US-10 EB near MN-301 Drive with ca...
...                                                  ...
45829  Closed at Sierra Way - Road closed due to acci...
45830   Closed at Main St - Road closed due to accident.
45831  Incident on PA-68 EB near STONEY LONESOME RD R...
45832  Closed at Roosevelt Blvd - Road closed due to ...
45833  Incident on US-30 WB near LC HEMPFIELD PT Road...

[45834 rows x 1 columns]
      Description
0               0
1               0
2               0
3               0
4               0
...           ...
45829           1
45830           1
45831           1
45832           1
45833           1

[45834 rows x 1 columns]


In [ ]:
#df_train[columnas_dia] = df_train[columnas_dia].apply(arreglar_dia).astype(float)
"""
df_train['Sunrise_Sunset'] = df_train['Sunrise_Sunset'].apply(arreglar_dia)
df_train['Civil_Twilight'] = df_train['Civil_Twilight'].apply(arreglar_dia)
df_train['Nautical_Twilight'] = df_train['Nautical_Twilight'].apply(arreglar_dia)
df_train['Astronomical_Twilight'] = df_train['Astronomical_Twilight'].apply(arreglar_dia)
"""
#print(df_train['Sunrise_Sunset'])

X = np.array(df_train[columnas_descripcion])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
print(df_test[columnas_lugar])

       Amenity   Bump  Crossing  Give_Way  Junction  No_Exit  Railway  \
0        False  False     False     False     False    False    False   
1        False  False     False     False     False    False    False   
2        False  False     False     False     False    False    False   
3        False  False     False     False     False    False    False   
4        False  False     False     False     False    False    False   
...        ...    ...       ...       ...       ...      ...      ...   
11367    False  False     False     False     False    False    False   
11368    False  False      True     False     False    False    False   
11369    False  False     False     False     False    False    False   
11370    False  False     False     False     False    False    False   
11371    False  False     False     False     False    False    False   

       Roundabout  Station   Stop  Traffic_Calming  Traffic_Signal  \
0           False    False  False            False   

In [ ]:
"""
# Tomamos un valor de alpha entre 10^-10 y 1. Usamos un barrido logaritmico.
for alpha in np.logspace(-10, 0, 11):

    # Creamos el modelo para un dado valor de alpha
    dtr = RandomForestClassifier(ccp_alpha = alpha)

    # Calculamos los scores al hacer cross-validation con 5 folds
    # Al especificar el valor de cv, cross_val_score hace cross-validation estratificado
    scores = cross_val_score(dtr, X_train, y_train, cv = 5)

    # Imprimimos el valor de alpha y el score medio
    print('Valor de alpha {} - Score medio = {}'.format(alpha, scores.mean()))
    """

Valor de alpha 1e-10 - Score medio = 0.686625355537446
Valor de alpha 1e-09 - Score medio = 0.6867188231695966
Valor de alpha 1e-08 - Score medio = 0.6848175260772799
Valor de alpha 1e-07 - Score medio = 0.6865629773936671
Valor de alpha 1e-06 - Score medio = 0.6881526606578457
Valor de alpha 1e-05 - Score medio = 0.6854096691029167
Valor de alpha 0.0001 - Score medio = 0.680952582895263
Valor de alpha 0.001 - Score medio = 0.6411806699551572
Valor de alpha 0.01 - Score medio = 0.6107595958677406
Valor de alpha 0.1 - Score medio = 0.6107595958677406
Valor de alpha 1.0 - Score medio = 0.6107595958677406


In [ ]:
df_graves = df_train[df_train.Severity==1]
df_leves = df_train[df_train.Severity==0]
print(df_graves['Description'].to_numpy())

['Incident on US-218 NB near COUNTY HWY-J56 Road closed. Take alternate route.'
 'Closed road from Leonard St NW (Alpine Ave NW) to Pannell Ave NW (Alpine Ave NW) due to accident.'
 'Closed at Dooling - Road closed due to accident.' ...
 'Incident on PA-68 EB near STONEY LONESOME RD Road closed. Take alternate route.'
 'Closed at Roosevelt Blvd - Road closed due to accident.'
 'Incident on US-30 WB near LC HEMPFIELD PT Road closed. Take alternate route.']


In [ ]:
clf = RandomForestClassifier()


clf.fit(X_train, y_train)


y_hat_test = clf.predict_proba(X_test)[:, 1]
y_hat_train = clf.predict_proba(X_train)[:, 1]

roc_test = roc_auc_score(y_test, y_hat_test)
roc_train = roc_auc_score(y_train, y_hat_train)

roc_test, roc_train
print(roc_test)
print(roc_train)

# knn
"""
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
y_test_knn = knn.predict(X_test)
y_train_knn = knn.predict(X_train)

roc_test = roc_auc_score(y_test, y_test_knn)
roc_train = roc_auc_score(y_train, y_train_knn)
roc_test, roc_train
"""

0.9632029924689927
0.9598512065841949


'\nknn = KNeighborsRegressor()\nknn.fit(X_train, y_train)\ny_test_knn = knn.predict(X_test)\ny_train_knn = knn.predict(X_train)\n\nroc_test = roc_auc_score(y_test, y_test_knn)\nroc_train = roc_auc_score(y_train, y_train_knn)\nroc_test, roc_train\n'

In [ ]:
X = np.array(df_test[columnas_clima])

predicciones = clf.predict_proba(X_test)[:, 1]

print(predicciones)


np.savetxt('predicciones.csv', predicciones, delimiter=",")

[0.73  0.44  0.332 ... 0.425 0.49  0.25 ]


In [ ]:
#np.savetxt('descripciones.csv', df_graves['Description'].to_numpy(), delimiter=',')
df_graves['Description'].to_excel('graves.xlsx')
df_leves['Description'].to_excel('leves.xlsx')

Mejoras:


*   n_estimators = 1000
*   mas features


*   closed
*   Elemento de la lista



